In [1]:
!pip install basedosdados

In [6]:
import basedosdados as bd

query = "SELECT * FROM datario.adm_central_atendimento_1746.chamado AS c WHERE DATE(c.data_inicio) = '2023-04-01'"
df = bd.read_sql(query, billing_project_id="emd-desafio-jr-data-scientist")



Downloading: 100%|██████████| 1756/1756 [00:00<00:00, 2429.72rows/s]


In [7]:
df

,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,data_real_diagnostico,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao
0,18516878,11,2023-04-01 10:49:16,2023-04-03 22:37:36,5,1,62026,119,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01
1,18518304,11,2023-04-01 20:54:06,2023-04-03 23:37:38,5,1,62323,95,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01
2,18518296,11,2023-04-01 20:48:40,2023-05-10 04:58:37,11,1,428706,15,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Atendido,None,1,2023-04-01
3,18517668,11,2023-04-01 15:49:36,2023-04-03 04:37:34,21,2,70698,43,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,1,2023-04-01
4,18518286,11,2023-04-01 20:37:06,2023-04-02 10:37:34,33,2,75101,28,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,18517285,1,2023-04-01 13:06:32,2023-04-01 15:48:27,43,3,34959,<NA>,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01
1752,18517321,1,2023-04-01 13:20:59,2023-04-01 15:49:07,24,2,80382,0,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01
1753,18518209,1,2023-04-01 19:55:08,2023-04-02 08:53:51,120,4,40931,<NA>,1706,TR/SUBTT/CGLF/CFV - Coordenadoria de Fiscaliza...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01
1754,18518101,11,2023-04-01 19:00:44,2023-04-10 22:46:27,89,3,27565,43,82,CLF - Coordenadoria de Licenciamento e Fiscali...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido parcialmente,None,0,2023-04-01


In [8]:
# prompt: #1 quantos chamados foram abertos no dia 01/04/2023?

df.shape[0]

1756

In [9]:
#2 qual o tipo de chamado que teve mais chamados abertos?

tipo_chamado = df.groupby('tipo').size().sort_values(ascending=False)
tipo_chamado.head(1)

,0
tipo,
Estacionamento irregular,366


In [10]:
#3 quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

query2 = "SELECT * FROM datario.dados_mestres.bairro AS b"
df2 = bd.read_sql(query2, billing_project_id="emd-desafio-jr-data-scientist")


Downloading: 100%|██████████| 164/164 [00:01<00:00, 145.44rows/s]


In [14]:
#fazer o merge com os valores IS NULL incluídos

chamado_bairro = df.merge(df2, left_on='id_bairro', right_on='id_bairro', how='left')

#3 bairros com mais chamados

chamado_bairro.groupby('nome').size().sort_values(ascending=False).head(3)

,0
nome,
Campo Grande,113
Tijuca,89
Barra da Tijuca,59


In [15]:
#4 qual o nome da subprefeitura com mais chamados abertos nesse dia?

subprefeitura = chamado_bairro.groupby('subprefeitura').size().sort_values(ascending=False)
subprefeitura.head(1)

,0
subprefeitura,
Zona Norte,510


In [16]:
#5 Existe algum chamado aberto que não foi associado a um bairro ou subprefeitura?

chamado_bairro.groupby('tipo').agg({'id_bairro': lambda x: x.isnull().sum(), 'subprefeitura': lambda x: x.isnull().sum()}).sort_values(by='id_bairro', ascending=False)

# Resposta: Sim, os tipos de chamados não são associados a bairro ou subprefeitura pois se tratam de chamados de atendimento a pessoas e problemas de transporte

,id_bairro,subprefeitura
tipo,,
Ônibus,50,50
Atendimento ao cidadão,18,18
Estacionamento irregular,1,1
BRT (corredor expresso de ônibus),1,1
Transporte Especial Complementar - TEC,1,1
Alvará,1,1
Defesa do Consumidor,1,1
Manejo Arbóreo,0,0
Manutenção de iluminação pública,0,0


In [17]:
#6 Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluíndo extremidades)

query3 = "SELECT * FROM datario.adm_central_atendimento_1746.chamado AS c WHERE c.subtipo = 'Perturbação do sossego' AND DATE(c.data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'"
df3 = bd.read_sql(query3, billing_project_id="emd-desafio-jr-data-scientist")



Downloading:   0%|          | 46247/11219508 [3:56:36<952:46:13,  3.26rows/s]

Downloading: 100%|██████████| 42830/42830 [00:11<00:00, 3612.70rows/s]


42830


In [18]:
print(df3.shape[0])

42830


In [19]:
#7 Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos

query4 = "SELECT * FROM datario.adm_central_atendimento_1746.chamado AS c LEFT JOIN datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos AS e ON DATE(c.data_inicio) = e.data_inicial WHERE c.subtipo = 'Perturbação do sossego' AND DATE(c.data_inicio) BETWEEN '2022-01-01' AND '2023-12-31' AND e.evento IS NOT NULL"
df4 = bd.read_sql(query4, billing_project_id="emd-desafio-jr-data-scientist")

Downloading: 100%|██████████| 295/295 [00:00<00:00, 1379.70rows/s]


In [20]:
df4.head()

,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,ano,data_inicial,data_final,evento,taxa_ocupacao
0,17661669,11,2022-09-02 22:12:04,2022-09-03 01:50:18,66,3,70144,557,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Atendido,None,0,2022-09-01,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
1,17659216,1,2022-09-02 10:37:27,2022-09-09 08:30:57,5,1,62331,405,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,1,2022-09-01,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
2,17661720,1,2022-09-02 23:04:08,2022-09-09 09:13:46,14,1,70235,518,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2022-09-01,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
3,17661683,1,2022-09-02 22:30:30,2022-09-09 09:08:30,25,2,82479,47,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2022-09-01,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
4,17661610,1,2022-09-02 21:20:15,2022-09-09 09:02:10,20,2,69500,165,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2022-09-01,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184


In [21]:
#8 quantos chamados por evento?

df4.groupby('evento').size().sort_values(ascending=False)

,0
evento,
Rock in Rio,177
Carnaval,74
Reveillon,44


In [22]:
#9 qual evento teve a maior média diária de chamados abertos?

# Calcula a duração do evento em dias, garantindo que a duração mínima seja 1 dia
df4['duracao_evento'] = (df4['data_final'] - df4['data_inicial']).dt.days + 1
df4['duracao_evento'] = df4['duracao_evento'].apply(lambda x: max(x, 1))

# Agrupa por evento e calcula o total de chamados e a duração total de cada evento
df_result = df4.groupby('evento').agg({
    'id_chamado': 'count',
    'duracao_evento': 'mean'}).reset_index()

# Calcula a média diária de chamados por evento
df_result['media_diaria_chamados'] = df_result['id_chamado'] / df_result['duracao_evento']
df_result = df_result.sort_values(by='media_diaria_chamados', ascending=False).reset_index(drop=True)

print(df_result.head(1))


        evento  id_chamado  duracao_evento  media_diaria_chamados
0  Rock in Rio         177        3.435028              51.527961
